In [1]:
import gc
import torch
import numpy as np

# Cuda maintenance
gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Torch device: ", device)

/home/rpc/Desktop/NIR/NIR/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch device:  cuda


----------------

## Prepare datasets

---

In [2]:
from custom_transformations import PerImageNormalization
from jpeg_dataset import JpegDataset
from torchvision import transforms
from labels import Label

In [4]:
from summer2winter_yosemite_metadata import Summer2WinterYosemiteMetadata

DATA_ROOT_DIR = '/home/nastya/Desktop/NIR/data/summer2winter_yosemite'
metadata = Summer2WinterYosemiteMetadata(DATA_ROOT_DIR)

In [3]:
from nordland_metadata import NordlandMetadata

DATA_ROOT_DIR = '/home/rpc/Desktop/nordland_data'
metadata = NordlandMetadata(DATA_ROOT_DIR)

In [4]:
# Dataset parameters
CLASS_NUM = metadata.get_num_classes()
IMG_SIZE = 256 # Image size required by AlexNet
IMG_CHANNELS = 3

In [5]:
image_transformation = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(IMG_SIZE),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        PerImageNormalization()])

train_dataset = torch.utils.data.ConcatDataset([
    JpegDataset(metadata.get_train_data(Label.Summer), image_transformation),
    JpegDataset(metadata.get_train_data(Label.Winter), image_transformation)])

test_dataset = torch.utils.data.ConcatDataset([
    JpegDataset(metadata.get_test_data(Label.Summer), image_transformation),
    JpegDataset(metadata.get_test_data(Label.Winter), image_transformation)])

In [6]:
from torch.utils.data import random_split

validation_set_size = int(0.2 * len(train_dataset))
train_set_size = len(train_dataset) - validation_set_size

train_subset, validation_subset = random_split(train_dataset, [train_set_size, validation_set_size],
                                               generator=torch.Generator().manual_seed(42))

-----------------

## Define architecture
-----------------


$$  o = \left \lfloor \frac{current\_image\_size + 2*padding\_size - filter\_size}{stride\_size} \right \rfloor + 1 $$


In [7]:
from alexnet.alexnet import AlexNet

In [8]:
model = AlexNet(CLASS_NUM, 0.5)

In [9]:
for layer in model.children():
    print("Layer : {}".format(layer))
    print("Parameters : ")
    for param in layer.parameters():
        print(param.shape)
    print()

Layer : Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Parameters : 
torch.Size([64, 3, 11, 11])
torch.Size([64])
torch.Size([192, 64, 5, 5])
torch.Size([192])
torch.Size([384, 192, 3, 3])
torch.Size([384])
torch.Size([256, 384, 3, 3])
torch.Size([256])
torch.Size([256, 256, 3, 3]

In [10]:
model = model.to(device=device) # send the model for training on either cuda or cpu

In [11]:
from torch.utils.data import DataLoader

BATCH_SIZE = 128

train_loader = DataLoader(train_subset, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(validation_subset, batch_size = BATCH_SIZE, shuffle = False)

In [12]:
from torch.optim import Adam
from alexnet.utils import train_model
from torch import nn

epochs = 30
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = learning_rate)

metrics = train_model(model, loss_fn, optimizer, train_loader, val_loader, device, epochs)

100%|████████████████████████████████████████████████████████████████████████████████████████| 308/308 [34:37<00:00,  6.74s/it]


Train loss : 0.106
Valid Loss : 0.027
Valid Acc  : 0.990


100%|████████████████████████████████████████████████████████████████████████████████████████| 308/308 [34:56<00:00,  6.81s/it]


Train loss : 0.020
Valid Loss : 0.015
Valid Acc  : 0.994


100%|████████████████████████████████████████████████████████████████████████████████████████| 308/308 [34:33<00:00,  6.73s/it]


...

Train loss : 0.038
Valid Loss : 0.249
Valid Acc  : 0.919


100%|████████████████████████████████████████████████████████████████████████████████████████| 308/308 [34:28<00:00,  6.72s/it]


Train loss : 0.086
Valid Loss : 0.019
Valid Acc  : 0.989


 83%|████████████████████████████████████████████████████████████████████████▊               | 255/308 [28:38<05:57,  6.74s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

plt.plot(metrics["train loss"], label = "train loss")
plt.plot(metrics["valid loss"], label = "valid loss")
plt.plot(metrics["valid acc"], label = "valid acc")
plt.legend()
plt.show()

In [15]:
from alexnet.utils import predict

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
y_actual, y_preds = predict(model, test_loader, device)

In [16]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Test Accuracy : {}".format(accuracy_score(y_actual, y_preds)))
print("\nClassification Report : ")
print(classification_report(y_actual, y_preds))
print("\nConfusion Matrix : ")
print(confusion_matrix(y_actual, y_preds))

Test Accuracy : 0.7129798903107861

Classification Report : 
              precision    recall  f1-score   support

           0       0.71      0.82      0.76       309
           1       0.71      0.58      0.64       238

    accuracy                           0.71       547
   macro avg       0.71      0.70      0.70       547
weighted avg       0.71      0.71      0.71       547


Confusion Matrix : 
[[253  56]
 [101 137]]
